# AI-Driven Optical PCB Inspection for Smartphone Assembly

Category: General problem

Description: Manual PCB inspections are time-consuming and error-prone. Students can develop a computer vision-based inspection tool to reduce inspection time and avoid costly field returns—especially critical in high-volume EMS environments.


# AI Project Lifecycle

![AI Lifecycle](./assets/lifecycle.png)


## Step 1: Problem Scoping

---------------------------

FILL ME

---------------------------


## Step 2: Data Acquisiton

Here, we have gathered data from the following sources, with different types of data:

These data sources detail defects on the copper tracks: Used to train Model CopperTrack
- DSPCBSD-.v1i.yolov11: https://universe.roboflow.com/pcb-egrla/dspcbsd/dataset/1
- PCB_DATASET: https://www.kaggle.com/datasets/akhatova/pcb-defects
- DeepPCB-master: https://github.com/tangsanli5201/DeepPCB
- MIXED PCB DEFECT DETECTION: https://data.mendeley.com/datasets/fj4krvmrr5/1

This dataset contains annotated images of PCBs with some missing soldered components:  It is not used as of now.
- PCB defects.v2i.yolov11: https://universe.roboflow.com/uni-4sdfm/pcb-defects/dataset/2

These datasets have annotated images of PCBs highlighting actual components, this can be used to identify missing components by comparing it with a known good PCB's output. This is also not used as of now.
- pcb-electronic-components-dataset: https://www.kaggle.com/datasets/rahul14112003/pcb-electronic-components-dataset
- pcb-oriented-detection: https://www.kaggle.com/datasets/yuyi1005/pcb-oriented-detection
- pcb-fault-detection: (Very Poor Quality) https://www.kaggle.com/datasets/animeshkumarnayak/pcb-fault-detection
- pcb-component-detection: https://sites.google.com/view/chiawen-kuo/home/pcb-component-detection

The nvidia folder has a pre-trained model from https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/pcb_classification, as they do not provide the dataset they used for training. 

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
import xml.etree.ElementTree as ET
from shutil import copyfile
import os
import os.path as path
import shutil
import pathlib
from pathlib import Path
from tqdm.std import tqdm
import random

In [ ]:
FINAL_DATA_DIR = Path("./final_track_data")
clear_old_data = False
if clear_old_data and path.exists(FINAL_DATA_DIR):
    shutil.rmtree(FINAL_DATA_DIR)
os.mkdir(FINAL_DATA_DIR)

In [3]:
CLASSES = [
    "Short",
    "Spur",
    "Spurious copper",
    "Open",
    "Mouse bite",
    "Hole breakout",
    "Conductor scratch",
    "Conductor foreign object",
    "Base material foreign object",
    "Missing hole",
]

SHORT = 0
SPUR = 1  # Extra copper protruding out of the copper track
SPURIOUS_COPPER = 2  # Extra copper outside of a track
OPEN = 3
MOUSE_BITE = 4  # Copper removed from the track
HOLE_BREAKOUT = 5  # Hole misaligned
SCRATCH = 6
CONDUCTOR_FOREIGN_OBJECT = 7
BASE_MATERIAL_FOREIGN_OBJECT = 8
MISSING_HOLE = 9

In [4]:
DATASET_GROUPS = ["train", "test", "valid"]
for g in DATASET_GROUPS:
    os.mkdir(FINAL_DATA_DIR / g)

In [5]:
# Define metadata for dataset
with open(FINAL_DATA_DIR / "data.yaml", "w") as f:
    f.write(
        f"""train: ../train/images
val: ../valid/images
test: ../test/images

nc: {len(CLASSES)}
names: [{",".join(f"'{s}'" for s in CLASSES)}]
"""
    )

#### Transfer files from one YOLO dataset to another, while mapping class names

In [31]:
def map_one_dataset_group(
    from_path_top: Path, to_path_top: Path, dataset_group: str, mapping: dict[int, int]
):
    from_path = from_path_top / dataset_group
    to_path = to_path_top / dataset_group
    shutil.copytree(from_path / "images", to_path / "images", dirs_exist_ok=True)
    to_labels_path = to_path / "labels"
    to_labels_path.mkdir(parents=True, exist_ok=True)
    for from_child_path in tqdm((from_path / "labels").iterdir()):
        to_child_path = to_labels_path / from_child_path.name
        with open(from_child_path, "r") as f:
            with open(to_child_path, "w") as t:
                for line in f:
                    class_id, x, y, w, h = line.strip().split()
                    new_class_id = str(mapping[int(class_id)])
                    t.write(f"{new_class_id} {x} {y} {w} {h}\n")


def map_dataset(from_path_top: Path, to_path_top: Path, mapping: dict[int, int]):
    for g in tqdm(DATASET_GROUPS):
        map_one_dataset_group(from_path_top, to_path_top, g, mapping)

#### Converting Pascal VOC Format of Dataset to YOLO Format

**Pascal VOC** : $(x_{min}, y_{min}, x_{max},y_{max})$

**YOLO** : $(x_{center-norm},y_{center-norm}, w_{norm}, h_{norm})$

$x_{norm}$ = $\frac{x}{widthofWholeImage}$

$y_{norm}$ = $\frac{y}{heightofWholeImage}$

$w_{norm}$ = $\frac{w}{widthofWholeImage}$

$h_{norm}$ = $\frac{h}{heightofWholeImage}$

In [8]:
def convert_bbox_to_yolo(
    size: tuple[int, int], box: tuple[int, int, int, int]
) -> tuple[int, int, int, int]:
    """Convert bounding box coordinates from PASCAL VOC format to YOLO format.

    :param size: A tuple of the image size: (width, height)
    :param box: A tuple of the PASCAL VOC bbox: (xmin, ymin, xmax, ymax)
    :return: A tuple of the YOLO bbox: (x_center, y_center, width, height)
    """
    # Calculate relative dimensions
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]

    # Calculate center, width, and height of the bbox in relative dimension
    rel_x_center = (box[0] + box[2]) / 2.0 * dw
    rel_y_center = (box[1] + box[3]) / 2.0 * dh
    rel_width = (box[2] - box[0]) * dw
    rel_height = (box[3] - box[1]) * dh

    return (rel_x_center, rel_y_center, rel_width, rel_height)

In [9]:
def xml_to_txt(input_file: Path, output_txt: Path, classes: dict[str, int]):
    """Parse an XML file in PASCAL VOC format and convert it to YOLO format.

    :param input_xml: Path to the input XML file.
    :param output_txt: Path to the output .txt file in YOLO format.
    :param classes: A list of class names as strings.
    """
    # Load and parse the XML file
    if input_file.suffix == ".txt":
        # Try to parse .txt as XML
        try:
            # Attempt to parse the file content as XML
            with input_file.open("r", encoding="utf-8") as file:
                file_content = file.read()
            root = ET.fromstring(file_content)
        except ET.ParseError as e:
            print(f"Error parsing {input_file}: {e}")
            return  # Skip this file and continue with the next
    else:
        # Try parsing the XML file (expects XML format)
        try:
            tree = ET.parse(input_file)
            root = tree.getroot()
        except ET.ParseError as e:
            print(f"Error parsing {input_file}: {e}")
            return  # Skip this file and continue with the next

    # Extract image dimensions
    size_element = root.find("size")
    image_width = int(size_element.find("width").text)
    image_height = int(size_element.find("height").text)

    with output_txt.open("w") as file:
        # Process each object in the XML
        for obj in root.iter("object"):
            is_difficult = obj.find("difficult").text
            class_name = obj.find("name").text

            # Skip "difficult" objects or if the name is not in classes
            if class_name not in classes or int(is_difficult) == 1:
                continue

            class_id = classes[class_name]

            # Extract and convert bbox
            xml_box = obj.find("bndbox")
            bbox = (
                float(xml_box.find("xmin").text),
                float(xml_box.find("ymin").text),
                float(xml_box.find("xmax").text),
                float(xml_box.find("ymax").text),
            )
            yolo_bbox = convert_bbox_to_yolo((image_width, image_height), bbox)

            # Write to the output file in YOLO format
            file.write(f"{class_id} {' '.join(map(str, yolo_bbox))}\n")

In [14]:
def map_pascal_to_yolo_one_dataset_group(
    files_list: list[tuple[Path, Path]],
    to_path_with_group: Path,
    mapping: dict[str, int],
):
    to_images_path = to_path_with_group / "images"
    to_labels_path = to_path_with_group / "labels"
    to_images_path.mkdir(parents=True, exist_ok=True)
    to_labels_path.mkdir(parents=True, exist_ok=True)
    for image, xml in tqdm(files_list):
        shutil.copy(image, to_images_path / image.name)
        to_child_path = to_labels_path / xml.with_suffix(".txt").name
        xml_to_txt(xml, to_child_path, mapping)


def map_pascal_to_yolo_dataset(
    files_list: list[tuple[Path, Path]], to_path_top: Path, mapping: dict[int, int]
):
    rng = random.Random(x=42)
    rng.shuffle(files_list)
    train_end_idx = int(len(files_list) * 0.7)
    test_end_idx = int(len(files_list) * (0.7 + 0.15))
    train_files = files_list[0:train_end_idx]
    test_files = files_list[train_end_idx:test_end_idx]
    val_files = files_list[test_end_idx:]
    vs = [train_files, test_files, val_files]
    for i, g in tqdm(enumerate(DATASET_GROUPS)):
        map_pascal_to_yolo_one_dataset_group(vs[i], to_path_top / g, mapping)

### DSPCBSD-.v1i.yolov11: DsPCBSD+ Dataset

From https://www.nature.com/articles/s41597-024-03656-8

Most comprehensive of all datasets, i.e. has most classes. So we have used its class names for the entire aggregated dataset.


In [21]:
DSCPBSD_MAP = {
    0: SHORT,
    1: SPUR,
    2: SPURIOUS_COPPER,
    3: OPEN,
    4: MOUSE_BITE,
    5: HOLE_BREAKOUT,
    6: SCRATCH,
    7: CONDUCTOR_FOREIGN_OBJECT,
    8: BASE_MATERIAL_FOREIGN_OBJECT,
}

In [32]:
map_dataset(Path("./DSPCBSD-.v1i.yolov11/"), FINAL_DATA_DIR, DSCPBSD_MAP)

5124it [00:54, 93.86it/s] 0<?, ?it/s]
759it [00:07, 97.47it/s] 00<02:00, 60.37s/it]
1484it [00:15, 94.33it/s] 4<00:33, 33.17s/it]
100%|██████████| 3/3 [01:34<00:00, 31.48s/it]


### MIXED PCB DEFECT DATASET

From: https://data.mendeley.com/datasets/fj4krvmrr5/2

Has similar data, but is labelled differently, so we need to map the labels.

In [36]:
MIXED_PCB_DEFECT_DATASET_MAPPING = {
    0: MISSING_HOLE,
    1: MOUSE_BITE,
    2: OPEN,
    3: SHORT,
    4: SPUR,
    5: SPURIOUS_COPPER,
}

In [ ]:
map_dataset(
    Path("./MIXED PCB DEFECT DETECTION/"),
    FINAL_DATA_DIR,
    MIXED_PCB_DEFECT_DATASET_MAPPING,
)

1720it [00:18, 92.26it/s] 0<?, ?it/s]
11it [00:00, 94.25it/s]0:21<00:43, 21.91s/it]
10it [00:00, 107.50it/s]:22<00:09,  9.19s/it]
100%|██████████| 3/3 [00:22<00:00,  7.45s/it]


### PCB_DATASET

From https://www.kaggle.com/datasets/akhatova/pcb-defects by The Open Lab on Human Robot Interaction of Peking University

It is in Pascal VOC format, so we must first convert it to YOLO format.

In [15]:
PCB_DATASET_MAPPING = {
    "missing_hole": MISSING_HOLE,
    "mouse_bite": MOUSE_BITE,
    "spurious_copper": SPURIOUS_COPPER,
    "short": SHORT,
    "spur": SPUR,
    "open_circuit": OPEN,
}

In [ ]:
PCB_DATASET_PATH = Path("./PCB_DATASET/")
files_list = []
for category in (PCB_DATASET_PATH / "images").iterdir():
    for image_file in category.iterdir():
        data_file = (
            PCB_DATASET_PATH
            / "Annotations"
            / category.name
            / (image_file.with_suffix(".xml").name)
        )
        files_list.append((image_file, data_file))

In [17]:
map_pascal_to_yolo_dataset(files_list, FINAL_DATA_DIR, PCB_DATASET_MAPPING)

100%|██████████| 104/104 [00:09<00:00, 11.06it/s]
3it [00:54, 18.03s/it]


### DeepPCB-master

From https://github.com/tangsanli5201/DeepPCB

It's images are black & white, so we are ignoring it for now.